# Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from math import sqrt
import matplotlib.pyplot as plt
import joblib
import os


2025-01-30 15:22:42.545052: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 15:22:42.547502: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 15:22:42.575648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-30 15:22:42.575675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-30 15:22:42.576573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [18]:
scalers_dir = 'scalers'
rl_models_dir = 'RL_models'

# Additional functions


In [4]:
def inverse_scale_pos(scaler_pos, scaledPos):

  pos = scaler_pos.inverse_transform(scaledPos)

  return pos

In [5]:
def inverse_scale_forces(scaler_fx, scaler_fy, scaled_fx, scaled_fy):
  force_x = scaler_fx.inverse_transform(scaled_fx)
  force_y = scaler_fy.inverse_transform(scaled_fy)

  return force_x, force_y

In [6]:
def create_pos_motion_list(pos_cmd):
  pos_motion_list = []
  index = 1
  for i in range(len(pos_cmd)):
    if i == 0:
      new_value = 0.0
      pos_motion_list.append(new_value)
      continue
    if (pos_cmd[i - 1]) != (pos_cmd[i]):
      new_value = 0.0
      pos_motion_list.append(new_value)
      index = 1
    else:
      new_value = index / 100
      pos_motion_list.append(new_value)
      index += 1

  return pos_motion_list


In [7]:
def preprocess_pred_data(dataset, n):

  window = 100

  def make_dataset(dataset, look_back=1):
      dataX = []
      for i in range(len(dataset) - look_back - 1):
          xset = []
          for j in range(n):
              input = dataset[i:(i + look_back), j]
              xset.append(input)


          dataX.append(xset)

      return np.array(dataX)

  look_back = window
  predX = make_dataset(dataset, look_back)

  return predX

# RL inverse model

In [8]:
import argparse
import os
import gym
from gym import spaces
from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [9]:
import os
# Check the current directory
os.getcwd()

'/home/hama6767/robosoft/ML-soft-fin-motion-RoboSoft2025/surrogate_model'

In [10]:
lstm_forcenet = tf.keras.models.load_model('pretrained_models/lstm_force_net.keras')
cnn_posnet = tf.keras.models.load_model('pretrained_models/cnn_pos_net.keras')

In [11]:
print(lstm_forcenet.input_shape)
print(cnn_posnet.input_shape)

(None, 3, 100)
(None, 2, 100)


## Define training env and parameters

In [14]:
scaler_pos = joblib.load(os.path.join(scalers_dir,'scaler_pos.pkl'))
scaler_predicted_pos = MinMaxScaler(feature_range=(-1.57, 1.57))
scaler_pos_cmd = joblib.load(os.path.join(scalers_dir,'scaler_pos_cmd.pkl'))
scaler_ang_vel = joblib.load(os.path.join(scalers_dir,'scaler_ang_vel.pkl'))
scaler_fx = joblib.load(os.path.join(scalers_dir,'scaler_fx.pkl'))
scaler_fy = joblib.load(os.path.join(scalers_dir,'scaler_fy.pkl'))

/home/hama6767/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
def plot_returns(returns, window_size=100):
    plt.figure(figsize=(12, 6))
    plt.plot(returns, label='Episode Return')

    if len(returns) >= window_size:
        rolling_mean = pd.Series(returns).rolling(window=window_size).mean()
        plt.plot(rolling_mean, label=f'Rolling Mean (window={window_size})', color='orange')

    plt.xlabel('Episode')
    plt.ylabel('Return')
    plt.title('Episode Returns Over Time')
    plt.legend()
    plt.show()

In [16]:
class LearningFinMovement(gym.Env):
    def __init__(self, show_plot=False):
        super(LearningFinMovement, self).__init__()
        self.show_plot = show_plot
        self.max_steps = 250

        self.posnet = cnn_posnet  # Load posnet
        self.lstm_forcenet = lstm_forcenet  # Load forcenet
        self.current_step = 0
        self.observation_dimension = 5
        self.keep_cmd_history = 3
        self.force_cmd_freq = 30
        self.current_episode_return = 0  # Track the return for the current episode
        self.episode_returns = []

        self.observation_dimension = self.observation_dimension + self.keep_cmd_history * 2

        self.action_space = spaces.Box(low=np.array([-1.0, -1.0]),
                                        high=np.array([1.0, 1.0]), dtype=np.float16) # [angle, angular_vel]
        self.oscillation_length = 5

        self.observation_space = spaces.Box(low=-float("inf"),
                                            high=float("inf"),
                                            shape=(self.observation_dimension,),
                                            dtype=np.float16)

        self.command_hz = 30  # original actuator and sensor Hz
        self.window = 100  # original actuator and sensor Hz
        self.buffer_length = 500
        self.reward_calculate_length = self.command_hz * self.oscillation_length
        self.timesteps = 0

        self.target_force_x_init = 2.0
        self.target_force_x = 2.0 # Set a random or predefined value
        self.target_force_y = 0
        self.target_force_y_init = 0
        self.target_range = 0

        #### PARAMETERS ####
        self.weight_x = 1.0
        self.weight_y = 1.0
        self.weight_c = 0.2

        self.cmd_pos_buffer = np.zeros(self.buffer_length)
        self.cmd_vel_buffer = np.zeros(self.buffer_length)
        self.pos_motion_list_buffer = np.zeros(self.buffer_length)

        self.pos_cmd_history = np.zeros(self.keep_cmd_history)
        self.ang_vel_history = np.zeros(self.keep_cmd_history)

    def pred_PosNet(self, df, cmd_pos, cmd_vel, pos_motion_list):

        scaled_cmd_pos = scaler_pos_cmd.transform(np.array(cmd_pos).reshape(-1, 1))
        scaled_cmd_vel = scaler_ang_vel.transform(np.array(cmd_vel).reshape(-1, 1))
        pos_motion_list_scaled = np.array(pos_motion_list).reshape(-1, 1)

        # Stack the features together along the "channel" axis (axis=1)
        dataset = np.hstack([scaled_cmd_pos, scaled_cmd_vel, pos_motion_list_scaled])
        scaled_features = preprocess_pred_data(dataset, 2)


        # Predict with posnet
        predictions = self.posnet.predict(scaled_features)
        position =[predictions[:, 0]]

        posPredict = inverse_scale_pos(scaler_pos, position)

        df_pred = pd.DataFrame()
        df_pred['cmd_pos'] = cmd_pos
        df_pred['cmd_vel'] = cmd_vel
        df_pred['pos_motion_list'] = pos_motion_list

        posPredict_full = np.concatenate((np.full((self.window + 1,), 0), posPredict.flatten()))

        df_pred['position'] = posPredict_full

        if self.show_plot:
          plt.figure(figsize=(21, 7))
          plt.plot(df_pred['cmd_pos'], label='cmd_pos')
          plt.plot(df_pred['position'], label='position')
          plt.legend()
          plt.show()



        return df_pred

    def pred_ForceNet(self, df, position, cmd_pos, cmd_vel, pos_motion_list):
        scaled_position = scaler_pos.transform(np.array(position).reshape(-1, 1)) # share the scaler
        scaled_cmd_pos = scaler_pos_cmd.transform(np.array(cmd_pos).reshape(-1, 1))
        scaled_cmd_vel = scaler_ang_vel.transform(np.array(cmd_vel).reshape(-1, 1))
        pos_motion_list = np.array(pos_motion_list).reshape(-1, 1)

        dataset = np.hstack((scaled_position, scaled_cmd_pos, scaled_cmd_vel))
        scaled_features = preprocess_pred_data(dataset, 3)

        predictions = self.lstm_forcenet.predict(scaled_features)
        force_x = [predictions[:, 0]]
        force_y = [predictions[:, 1]]
        force_x, force_y = inverse_scale_forces(scaler_fx, scaler_fy, force_x, force_y)

        fx_predict_full = np.concatenate((np.full((self.window + 1,), 0), force_x.flatten()))
        fy_predict_full = np.concatenate((np.full((self.window + 1,), 0), force_y.flatten()))


        df_final = pd.DataFrame()
        df_final['position'] = position
        df_final['force_x'] = fx_predict_full
        df_final['force_y'] = fy_predict_full


        fx_average = np.mean(df_final['force_x'][-self.reward_calculate_length:])
        fy_average = np.mean(df_final['force_y'][-self.reward_calculate_length:])
        print("fx_average: ", fx_average)
        print("fy_average: ", fy_average)
        if self.show_plot:
          plt.figure(figsize=(21, 7))
          plt.plot(df_final['force_y'], label='fy')
          plt.axhline(y=self.target_force_x, color='r', linestyle='--', label='target_force_x')
          plt.plot(df_final['force_x'], label='fx')
          plt.axhline(y=fx_average, color='g', label='fx average')
          plt.axhline(y=self.target_force_y, color='b', linestyle='--', label='target_force_y')
          plt.axhline(y=fy_average, color='y', label='fy average')
          plt.legend()
          plt.show()


        return df_final

    def generate_oscillation(self, phase, amp, zd):
        next_pos = zd + amp/2 * self.phase
        if next_pos > 1.57:
            next_pos = 1.57
        elif next_pos < -1.57:
            next_pos = -1.57
        self.phase *= -1
        return next_pos

    def motor_forward_model(self, cmd_pos, cmd_vel, pos_motion_list):
        df_tmp = pd.DataFrame(columns=['cmd_pos', 'cmd_vel', 'position', 'pos_motion_list'])

        _cmd_pos = cmd_pos
        _cmd_vel = cmd_vel
        _pos_motion_list = pos_motion_list

        df_tmp['cmd_pos'] = _cmd_pos
        df_tmp['cmd_vel'] = _cmd_vel
        df_tmp['pos_motion_list'] = _pos_motion_list
        df_pred = self.pred_PosNet(df_tmp, df_tmp['cmd_pos'], df_tmp['cmd_vel'], df_tmp['pos_motion_list'])

        df_final = self.pred_ForceNet(df_pred, df_pred['position'], df_pred['cmd_pos'], df_pred['cmd_vel'], df_pred['pos_motion_list'])
        df_final = df_final.iloc[- self.reward_calculate_length:]

        return df_final

    def reset(self):
        self.timesteps = 0
        self.current_step = 0
        self.current_episode_return = 0
        self.reward_list = []
        self.cmd_pos_buffer = np.zeros(self.buffer_length)
        self.cmd_vel_buffer = np.zeros(self.buffer_length)
        self.pos_motion_list_buffer = np.zeros(self.buffer_length)
        obs = [0, self.target_force_x, self.target_force_y, 0, 0]
        if self.keep_cmd_history > 0:
          obs = np.array(obs)
          obs = np.concatenate((obs, self.pos_cmd_history, self.ang_vel_history))
          obs = obs.flatten()
        print("-----------RESET!!!!!!!!!!!!!-----------")
        return obs

    def step(self, action):
        start_time = time.perf_counter()

        self.current_step += 1

        _action_pos, _action_vel = action
        action_pos = _action_pos * 1.20
        action_vel = (_action_vel + 1.0)* 1.0 + 1.0
        print("action_pos: ", action_pos)
        print("action_vel: ", action_vel)
        self.pos_cmd_history = np.append(self.pos_cmd_history, action_pos)
        self.ang_vel_history = np.append(self.ang_vel_history, action_vel)

        for i in range(self.command_hz):
            self.cmd_pos_buffer = np.append(self.cmd_pos_buffer, action_pos)
            self.cmd_vel_buffer = np.append(self.cmd_vel_buffer, action_vel)

        self.cmd_pos_buffer = self.cmd_pos_buffer[-self.buffer_length:]
        self.cmd_vel_buffer = self.cmd_vel_buffer[-self.buffer_length:]

        self.pos_motion_list_buffer = create_pos_motion_list(self.cmd_pos_buffer)

        df_tmp = self.motor_forward_model(self.cmd_pos_buffer, self.cmd_vel_buffer, self.pos_motion_list_buffer)
        self.df_tmp = df_tmp

        self.pos_cmd_history = self.pos_cmd_history[-self.keep_cmd_history:]
        self.ang_vel_history = self.ang_vel_history[-self.keep_cmd_history:]

        # Calculate average force
        average_force_x = np.mean(df_tmp['force_x'])
        average_force_y = np.mean(df_tmp['force_y'])
        latest_force_x = np.mean(df_tmp['force_x'][-self.command_hz:-1])

        episode_done = self.current_step == self.max_steps
        reward = self.reward_function_s(average_force_x, average_force_y, self.target_force_x, self.target_force_y)
        self.current_episode_return += reward

        if episode_done:
            self.episode_returns.append(self.current_episode_return)
            self.current_episode_return = 0

        # Update target force before action to reflect on the observation
        if self.timesteps % self.force_cmd_freq == 0:
            self.target_force_x = self.target_force_x_init + self.target_range * np.random.uniform(-1, 1)
            self.target_force_y = self.target_force_y_init + self.target_range * np.random.uniform(-1, 1)


        latest_pos = df_tmp['position'].iloc[-1]
        # First, observe the environment
        observation = [latest_pos, self.target_force_x, self.target_force_y, action[0], action[1]]
        if self.keep_cmd_history > 0:
            observation = np.concatenate((observation, self.pos_cmd_history, self.ang_vel_history))
            observation = observation.flatten()

        print("observation: ", observation)

        self.timesteps += 1
        print("timesteps: ", self.timesteps)

        info = {}

        return observation, reward, episode_done, info

    def reward_function(self, force_x, force_y, target_force_x, target_force_y):
        reward = 0.0
        valid_error = 2.0

        force_error_x = force_x - target_force_x
        force_error_y = force_y - target_force_y

        reward_x = self.weight_x * -abs(force_error_x)
        reward_y = self.weight_y * -abs(force_error_y)

        reward += 4 + reward_x + reward_y

        print("target_force_x: ", target_force_x)
        print("target_force_y: ", target_force_y)
        print("force_error_x: ", force_error_x)
        print("force_error_y: ", force_error_y)
        print("REWARD: ", reward)
        print("--------")

        return reward

    def reward_function_s(self, force_x, force_y, target_force_x, target_force_y):
        reward = 0.0

        force_error_x = np.mean(force_x) - target_force_x
        force_error_y = np.mean(force_y) - target_force_y
        reward_x = self.weight_x * -abs(force_error_x)
        reward_y = self.weight_y * -abs(force_error_y)

        lambda_smoothness = 0.10
        force_x_diff = np.diff(self.df_tmp['force_x'][-self.reward_calculate_length:])
        force_y_diff = np.diff(self.df_tmp['force_y'][-self.reward_calculate_length:])
        smoothness_x = np.sum(force_x_diff**2)/len(force_x_diff)
        smoothness_y = np.sum(force_y_diff**2)/len(force_y_diff)
        sobolev_term = (lambda_smoothness * (smoothness_x + smoothness_y))**0.5

        reward = reward_x + reward_y - sobolev_term
        print("target_force_x: ", target_force_x)
        print("target_force_y: ", target_force_y)
        print("force_error_x: ", force_error_x)
        print("force_error_y: ", force_error_y)
        print("sobolev_term: ", sobolev_term)
        print("REWARD: ", reward)
        print("--------")

        return reward

## Agent Training

In [ ]:
import warnings
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO

# Define the environment
env = LearningFinMovement(False)


# Path where the checkpoints are saved
checkpoint_dir = rl_models_dir
checkpoint_prefix = 'y_ppo_fin_movement'

def find_latest_checkpoint(directory, prefix):
    checkpoints = [f for f in os.listdir(directory) if f.startswith(prefix)]

    if not checkpoints:
        return None
    checkpoints.sort(key=lambda x: int(x.split('_')[4].split('.')[0]), reverse=True)
    return os.path.join(directory, checkpoints[0])

latest_checkpoint = find_latest_checkpoint(checkpoint_dir, checkpoint_prefix)

if latest_checkpoint:
    print(f"Loading model from checkpoint: {latest_checkpoint}")
    model = RecurrentPPO.load(latest_checkpoint, env=env, n_steps=32, gamma=0.8) # ent_coef=0.001
else:
    print("No checkpoint found. Starting from scratch.")
    model = RecurrentPPO("MlpLstmPolicy", env, verbose=1, n_steps=32, gamma=0.8)
    policy_kwargs = dict(
                lstm_hidden_size=64,
                n_lstm_layers=3,
                net_arch=[64]
            )


checkpoint_callback = CheckpointCallback(save_freq=1000, save_path=checkpoint_dir, name_prefix=checkpoint_prefix)
warnings.filterwarnings("ignore", message="X does not have valid feature names")
model.learn(total_timesteps=5000, callback=checkpoint_callback)  # Adjust the total timesteps as needed
plot_returns(env.episode_returns)

model.save("final_model_ppo_fin_movement")

No checkpoint found. Starting from scratch.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------RESET!!!!!!!!!!!!!-----------
action_pos:  -0.29615428447723385
action_vel:  1.77028027176857


/home/hama6767/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


13/13 [==============================] - 0s 2ms/step
fx_average:  0.7304031928504545
fy_average:  -0.39518607911206793
target_force_x:  2.0
target_force_y:  0
force_error_x:  -1.2695968071495454
force_error_y:  -0.39518607911206793
sobolev_term:  0.10410959759505904
REWARD:  -1.7688924838566722
--------
observation:  [-0.14656989  2.          0.         -0.24679524 -0.22971973  0.
  0.         -0.29615428  0.          0.          1.77028027]
timesteps:  1
action_pos:  0.1996587038040161
action_vel:  1.0
13/13 [==============================] - 0s 2ms/step
fx_average:  0.8059635022658024
fy_average:  -0.5646425920303431
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.1940364977341975
force_error_y:  -0.5646425920303431
sobolev_term:  0.1519176000422859
REWARD:  -1.9105966898068265
--------
observation:  [ 0.04527545  2.          0.          0.16638225 -1.          0.
 -0.29615428  0.1996587   0.          1.77028027  1.        ]
timesteps:  2
action_pos:  -1.2
action_vel:  2

13/13 [==============================] - 0s 2ms/step
fx_average:  2.780629588985239
fy_average:  -0.6626616040043584
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  0.780629588985239
force_error_y:  -0.6626616040043584
sobolev_term:  0.41567669853032657
REWARD:  -1.8589678915199241
--------
observation:  [-0.24930018  2.          0.          0.906865   -0.5122816   0.62921126
 -1.2         1.088238    2.43255094  1.60577375  1.4877184 ]
timesteps:  16
action_pos:  -1.2
action_vel:  2.102526903152466
13/13 [==============================] - 0s 1ms/step
fx_average:  3.3417734142037534
fy_average:  -0.5566176549573231
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  1.3417734142037534
force_error_y:  -0.5566176549573231
sobolev_term:  0.4595798225644103
REWARD:  -2.357970891725487
--------
observation:  [-0.16921186  2.          0.         -1.          0.1025269  -1.2
  1.088238   -1.2         1.60577375  1.4877184   2.1025269 ]
timesteps:  17
action_pos:  -0.0492345526

13/13 [==============================] - 0s 5ms/step
fx_average:  2.0518504983211994
fy_average:  -0.9780850308507107
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  0.05185049832119937
force_error_y:  -0.9780850308507107
sobolev_term:  0.2996535474716668
REWARD:  -1.329589076643577
--------
observation:  [ 0.47209617  2.          0.          0.4890404   1.         -1.2
  0.64543405  0.58684849  1.42440343  2.94837606  3.        ]
timesteps:  31
action_pos:  0.12658288478851318
action_vel:  3.0
13/13 [==============================] - 0s 2ms/step
fx_average:  2.6670145581550706
fy_average:  -1.0418637256157828
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  0.6670145581550706
force_error_y:  -1.0418637256157828
sobolev_term:  0.3378550359809059
REWARD:  -2.0467333197517594
--------
observation:  [0.14169997 2.         0.         0.10548574 1.         0.64543405
 0.58684849 0.12658288 2.94837606 3.         3.        ]
timesteps:  32
---------------------------
| time

...

action_pos:  -1.2
action_vel:  2.672030448913574
13/13 [==============================] - 0s 3ms/step
fx_average:  0.8408233400070634
fy_average:  0.04367767972921999
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.1591766599929367
force_error_y:  0.04367767972921999
sobolev_term:  0.0897577504832785
REWARD:  -1.2926120902054352
--------
observation:  [-1.33079817  2.          0.         -1.          0.67203045 -1.2
 -1.2        -1.2         3.          3.          2.67203045]
timesteps:  167
action_pos:  -1.2
action_vel:  3.0
13/13 [==============================] - 0s 2ms/step
fx_average:  0.8698865613819089
fy_average:  0.04034231063950203
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.130113438618091
force_error_y:  0.04034231063950203
sobolev_term:  0.07808813491951848
REWARD:  -1.2485438841771115
--------
observation:  [-1.25287151  2.          0.         -1.          1.         -1.2
 -1.2        -1.2         3.          2.67203045  3.        ]
timestep

action_pos:  -1.2
action_vel:  3.0
13/13 [==============================] - 0s 2ms/step
fx_average:  0.9581066487464394
fy_average:  0.30499366622718194
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.0418933512535606
force_error_y:  0.30499366622718194
sobolev_term:  0.16537860601904703
REWARD:  -1.5122656234997895
--------
observation:  [-1.2319786  2.         0.        -1.         1.        -1.2
 -1.2       -1.2        3.         3.         3.       ]
timesteps:  181
action_pos:  -1.2
action_vel:  2.3669625520706177
13/13 [==============================] - 0s 2ms/step
fx_average:  1.3266557618497081
fy_average:  0.3345003981180536
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -0.6733442381502919
force_error_y:  0.3345003981180536
sobolev_term:  0.1832819577017343
REWARD:  -1.19112659397008
--------
observation:  [-1.45600957  2.          0.         -1.          0.36696255 -1.2
 -1.2        -1.2         3.          3.          2.36696255]
timesteps:  182
acti

13/13 [==============================] - 0s 2ms/step
fx_average:  0.3936338366690073
fy_average:  0.30409769176489426
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.6063661633309927
force_error_y:  0.30409769176489426
sobolev_term:  0.14016182108518693
REWARD:  -2.050625676181074
--------
observation:  [-1.23201615  2.          0.         -1.          0.82215047 -1.2
 -1.2        -1.2         3.          3.          2.82215047]
timesteps:  196
action_pos:  -1.2
action_vel:  1.8149963021278381
13/13 [==============================] - 0s 2ms/step
fx_average:  1.0147794395639855
fy_average:  0.3196796407102739
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -0.9852205604360145
force_error_y:  0.3196796407102739
sobolev_term:  0.19716773108237948
REWARD:  -1.5020679322286679
--------
observation:  [-1.57705737  2.          0.         -1.         -0.1850037  -1.2
 -1.2        -1.2         3.          2.82215047  1.8149963 ]
timesteps:  197
action_pos:  -1.2
action_ve

13/13 [==============================] - 0s 2ms/step
fx_average:  1.9973034210601122
fy_average:  1.0274746100405951
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -0.0026965789398878393
force_error_y:  1.0274746100405951
sobolev_term:  0.4630222231487732
REWARD:  -1.4931934121292563
--------
observation:  [-1.37555543  2.          0.         -1.         -0.62995291 -0.80403399
 -1.2        -1.2         2.70675093  1.87945169  1.37004709]
timesteps:  211
action_pos:  -1.2
action_vel:  3.0
13/13 [==============================] - 0s 2ms/step
fx_average:  2.002841871127112
fy_average:  0.9338333459599357
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  0.0028418711271118724
force_error_y:  0.9338333459599357
sobolev_term:  0.3865849679013023
REWARD:  -1.3232601849883499
--------
observation:  [-0.74615239  2.          0.         -1.          1.         -1.2
 -1.2        -1.2         1.87945169  1.37004709  3.        ]
timesteps:  212
----------------------------------

13/13 [==============================] - 0s 2ms/step
fx_average:  0.522200145508105
fy_average:  0.029842266903342813
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.477799854491895
force_error_y:  0.029842266903342813
sobolev_term:  0.028794733008576847
REWARD:  -1.536436854403815
--------
observation:  [-1.2319786  2.         0.        -1.         1.        -1.2
 -1.2       -1.2        3.         3.         3.       ]
timesteps:  224
action_pos:  -1.2
action_vel:  3.0
13/13 [==============================] - 0s 2ms/step
fx_average:  0.4089609663484751
fy_average:  0.04008759229404822
target_force_x:  2.0
target_force_y:  0.0
force_error_x:  -1.591039033651525
force_error_y:  0.04008759229404822
sobolev_term:  0.015877786674099455
REWARD:  -1.6470044126196728
--------
observation:  [-1.2319786  2.         0.        -1.         1.        -1.2
 -1.2       -1.2        3.         3.         3.       ]
timesteps:  225
action_pos:  -1.2
action_vel:  2.2613015174865723
13/13 [==